In [1]:
""" Repeating MNIST first experiment of swish paper. """

import numpy as np
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# For adding new activation function
from keras import backend as K
from keras.datasets import mnist
from keras.utils.generic_utils import get_custom_objects
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", Y_train.shape)

X_train original shape (60000, 28, 28)
y_train original shape (60000,)


In [3]:
# Normalization
nb_classes = 10
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [5]:
# Set the random seed
random_seed = 2

In [6]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [7]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.reshape(-1,784)
X_val = X_val.reshape(-1,784)
X_test = X_test.reshape(-1,784)
# test = test.values.reshape(-1,28,28,1)
print(X_train.shape, X_val.shape, X_test.shape)

(54000, 784) (6000, 784) (10000, 784)


In [18]:
def swish(x):
    return x*K.sigmoid(x)

def e_swish_2(x):
    sigmoid = K.sigmoid(x)
    return K.maximum(x*sigmoid, x*(2-sigmoid))

In [9]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
def create(act, n):
    model = Sequential()
    # First conv block
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation(act))
    for i in range(n-1):
        model.add(Dense(512))
        model.add(Activation(act))
        model.add(Dropout(0.3))
        
    model.add(Dense(nb_classes))
    model.add(Activation("sigmoid"))
    
    return model

In [16]:
record = []
for n in [10, 15, 20]:
    opt = SGD()
    # Set a learning rate annealer
#         learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.7, min_lr=0.00001) 
    # Common params 
    epochs = 10
    batch_size = 100
    # Create and compile the model
    model = model = create("relu", n)
#     model.summary()
    # Compile the model
    model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
    # Train the model
    history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                        verbose = 1)# , callbacks=[learning_rate_reduction])

    record.append([n, model.evaluate(X_test, Y_test)])
    print(n, ":", model.evaluate(X_test, Y_test))
    K.get_session().close()
    K.set_session(K.tf.Session())

print()
print()
print()
for r in record:
    print(r)




[10, [0.13656429858431221, 0.96499999999999997]]
[15, [1.1838343352317811, 0.43409999999999999]]
[20, [2.544732809829712, 0.12379999999999999]]


In [17]:
record = []
for n in [10, 15, 20]:
    opt = SGD()
    # Set a learning rate annealer
#         learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.7, min_lr=0.00001) 
    # Common params 
    epochs = 10
    batch_size = 100
    # Create and compile the model
    model = model = create(e_swish_2, n)
#     model.summary()
    # Compile the model
    model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
    # Train the model
    history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                        verbose = 1)# , callbacks=[learning_rate_reduction])

    record.append([n, model.evaluate(X_test, Y_test)])
    print(n, ":", model.evaluate(X_test, Y_test))
    K.get_session().close()
    K.set_session(K.tf.Session())

    
print()
print()
print()
for r in record:
    print(r)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 32s 600us/step - loss: 1.2627 - acc: 0.5371 - val_loss: 0.3269 - val_acc: 0.9152
Epoch 2/10
54000/54000 [==============================] - 31s 570us/step - loss: 0.3505 - acc: 0.8965 - val_loss: 0.1973 - val_acc: 0.9457
Epoch 3/10
54000/54000 [==============================] - 31s 571us/step - loss: 0.2379 - acc: 0.9323 - val_loss: 0.1671 - val_acc: 0.9557
Epoch 4/10
54000/54000 [==============================] - 31s 571us/step - loss: 0.1883 - acc: 0.9463 - val_loss: 0.1348 - val_acc: 0.9633
Epoch 5/10
54000/54000 [==============================] - 31s 574us/step - loss: 0.1596 - acc: 0.9549 - val_loss: 0.1179 - val_acc: 0.9672
Epoch 6/10
54000/54000 [==============================] - 31s 579us/step - loss: 0.1392 - acc: 0.9615 - val_loss: 0.1103 - val_acc: 0.9707
Epoch 7/10
54000/54000 [==============================] - 31s 573us/step - loss: 0.1217 - acc: 0.9657 - val_loss: 0.1

In [19]:
record = []
for n in [10, 15, 20]:
    opt = SGD()
    # Set a learning rate annealer
#         learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.7, min_lr=0.00001) 
    # Common params 
    epochs = 10
    batch_size = 100
    # Create and compile the model
    model = model = create(swish, n)
#     model.summary()
    # Compile the model
    model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
    # Train the model
    history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
                        verbose = 1)# , callbacks=[learning_rate_reduction])

    record.append([n, model.evaluate(X_test, Y_test)])
    print(n, ":", model.evaluate(X_test, Y_test))
    K.get_session().close()
    K.set_session(K.tf.Session())

    
print()
print()
print()
for r in record:
    print(r)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 33s 614us/step - loss: 2.3019 - acc: 0.1122 - val_loss: 2.3016 - val_acc: 0.1152
Epoch 2/10
54000/54000 [==============================] - 28s 512us/step - loss: 2.3013 - acc: 0.1121 - val_loss: 2.3013 - val_acc: 0.1152
Epoch 3/10
54000/54000 [==============================] - 28s 513us/step - loss: 2.3010 - acc: 0.1121 - val_loss: 2.3012 - val_acc: 0.1152
Epoch 4/10
54000/54000 [==============================] - 29s 533us/step - loss: 2.3009 - acc: 0.1121 - val_loss: 2.3011 - val_acc: 0.1152
Epoch 5/10
54000/54000 [==============================] - 28s 526us/step - loss: 2.3008 - acc: 0.1121 - val_loss: 2.3010 - val_acc: 0.1152
Epoch 6/10
54000/54000 [==============================] - 29s 531us/step - loss: 2.3007 - acc: 0.1121 - val_loss: 2.3009 - val_acc: 0.1152
Epoch 7/10
54000/54000 [==============================] - 29s 531us/step - loss: 2.3006 - acc: 0.1121 - val_loss: 2.3

KeyboardInterrupt: 

In [10]:
# results = []
# for act in ["relu", e_swish_2, swish]:
#     record = []
#     for n in [15, 20, 25, 30]:
#         opt = SGD()
#         # Set a learning rate annealer
# #         learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.7, min_lr=0.00001) 
#         # Common params 
#         epochs = 10
#         batch_size = 100
#         # Create and compile the model
#         model = model = create("relu", 5)
#     #     model.summary()
#         # Compile the model
#         model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
#         # Train the model
#         history = model.fit(X_train,Y_train, epochs = epochs, validation_data = (X_val,Y_val),
#                             verbose = 0)# , callbacks=[learning_rate_reduction])

#         record.append([n, model.evaluate(X_test, Y_test)])
#         print(n, ":", model.evaluate(X_test, Y_test))
        
#     results.append(record)
    
#     print()
#     print()
#     print()

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 17s 318us/step - loss: 1.3883 - acc: 0.5310 - val_loss: 0.3938 - val_acc: 0.8885
Epoch 2/10
54000/54000 [==============================] - 16s 298us/step - loss: 0.3980 - acc: 0.8826 - val_loss: 0.2486 - val_acc: 0.9253
Epoch 3/10
54000/54000 [==============================] - 16s 298us/step - loss: 0.2686 - acc: 0.9203 - val_loss: 0.1871 - val_acc: 0.9410
Epoch 4/10
54000/54000 [==============================] - 16s 298us/step - loss: 0.2072 - acc: 0.9389 - val_loss: 0.1535 - val_acc: 0.9522
Epoch 5/10
54000/54000 [==============================] - 16s 298us/step - loss: 0.1737 - acc: 0.9489 - val_loss: 0.1416 - val_acc: 0.9557
Epoch 6/10
54000/54000 [==============================] - 16s 299us/step - loss: 0.1446 - acc: 0.9590 - val_loss: 0.1265 - val_acc: 0.9610
Epoch 7/10
54000/54000 [==============================] - 16s 299us/step - loss: 0.1263 - acc: 0.9622 - val_loss: 0.1

Epoch 4/10
54000/54000 [==============================] - 16s 297us/step - loss: 0.2077 - acc: 0.9397 - val_loss: 0.1505 - val_acc: 0.9535
Epoch 5/10
54000/54000 [==============================] - 16s 296us/step - loss: 0.1726 - acc: 0.9479 - val_loss: 0.1328 - val_acc: 0.9582
Epoch 6/10
54000/54000 [==============================] - 16s 297us/step - loss: 0.1489 - acc: 0.9560 - val_loss: 0.1225 - val_acc: 0.9612
Epoch 7/10
54000/54000 [==============================] - 16s 297us/step - loss: 0.1257 - acc: 0.9632 - val_loss: 0.1154 - val_acc: 0.9637
Epoch 8/10
54000/54000 [==============================] - 16s 296us/step - loss: 0.1143 - acc: 0.9654 - val_loss: 0.1068 - val_acc: 0.9675
Epoch 9/10
54000/54000 [==============================] - 16s 298us/step - loss: 0.1007 - acc: 0.9698 - val_loss: 0.0956 - val_acc: 0.9683
Epoch 10/10
10000/10000 [==============================] - 1s 77us/step
8 : [0.08903080306346528, 0.97330000000000005]
Train on 54000 samples, validate on 6000 sample

In [ ]:
probs_1 = model.predict_proba(X_test)
scores = model.evaluate(X_test, Y_test)
print(scores)